# Neural Ordinary Differential Equations

## Introduction and Motivation

### What is an ODE?

* Vertical Projectile motion:


$$\sum{F} = \frac{dp}{dt} $$ 

$$- (F_{drag} + F_{grav}) = \frac{dp}{dt} $$ 

$$ -(k \frac{dy}{dt} + m g) = m \frac{d^2y}{dt^2} $$ 
    
By integrating both sides from $ t_0 $ to $ t_0 + t $:
        
$$ -(k y + m g t) = m \frac{dy}{dt} $$ 

$$ \frac{dy}{dt} = \frac{-(k y + m g t)}{m} \tag{1.1}$$ 
$$ \frac{dy}{dt} = f(y, t) \tag{1.2}$$ 

One way of numerically solving $(1.2)$, would be by integrating in $t$ with discrete steps of $t$:

$$ \frac{dy}{dt} = f(y, t) \tag{1.2}$$ 

$$y(t_0 + \Delta t) - y(t_0) = \Delta t   f(y(t_0), t)$$

$$y(t_0 + 2\Delta t) - y(t_0 + \Delta t) = \Delta t   f(y(t_0 + \Delta t), t)$$

$$y(t_0 + 3\Delta t) - y(t_0 + 2\Delta t) = \Delta t   f(y(t_0 + 2\Delta t), t)$$


$$...$$


$$ y(t_n + \Delta t)  = \Delta t f(y(t_n), t) + y(t_n) \tag{1.3}$$


* The previous steps are known as `Euler's method`.

* It's known for some time, that that $(1.3)$ resembles very much with the characteristic equation of `ResNets`, where $f(y_n, t)$ represents the output of a layer $n$ given $y_n$ as input.

### The classic ResNet example

- ResNets had the best accuracy in the ImageNet Competition (2015)
- ResNets uses skip-connections between layers, so the "depth" of the network becomes a feature to be learnt 
- Can have up to 100 layers while avoiding vanishing gradients.
<img src="assets/1.png">
src: https://arxiv.org/abs/1512.03385

### How it works?

* Instead of using feeding the output of a previous layer into the next layer:
$$
f(z(t-1),\ \theta(t-1)) = z(t) $$

$$
f(z(t),\ \theta(t)) = z(t+1)
$$

* We feed the input of the previous layer as well:
$$
f(z(t-1),\ \theta(t-1)) + z(t-1)= z(t) $$

$$
f(z(t),\ \theta(t)) + z(t)= z(t+1)
$$

* Thus, if the network with $\theta$ as parameters is trained with a set of measurements $\{(z_0, t_0),(z_1, t_1),...,(z_M, t_M)\}$, it will approximate the dynamics function $f(\theta, z)$

## Neural Ordinary Differential Equations

* Given the noisy measurements:


$$\{(z_0, t_0),(z_1, t_1),...,(z_M, t_M)\}$$


* One wants to find an approximation $\hat{f}(z, t, \theta)$ for the dynamics of $f(z, t)$ where

$$ \frac{\partial z}{\partial t}  = f(z(t), t, \theta)$$ or even $$z(t + \delta t) = \int_{t}^{t + \delta t} f(z(t), t, \theta)dt + z(t) \tag{2}$$


* Given $(z_0, t_0), (z_1, t_1)$, The system begin evolving and from $z_0, t_0$ until it reaches $z_1, t_0$. 


* One approximation ($\hat{z_1}, t_1$) of the $\hat{f}(z, t, \theta)$ would be achieved through an integration of $(1)$


$$\Big( \int_{t_0}^{t_1} f(z(t), t, \theta)dt = \hat{z}(t_1)\Big)$$

* In the case where a optimizer wants to approximate $\hat{z}(t)$, a cost function would be defined:
$$L(z(t_1)) = L \Big( \int_{t_0}^{t_1} f(z(t), t, \theta)dt \Big) = L \big( \text{ODESolve}(z(t_0), f, t_0, t_1, \theta) \big)$$


* One main difference between optimizing $\theta$ so $\hat{f}(z, t, \theta)$ can approximate the dynamics and the mere backpropagation of the $L(z(t_1))$ through a discrete number of layers, is that:


    - It will be used only one "layer-like network" with the time as parameter (or the same weight distributed over time).
    
    - Backpropagation will have to propagate over infinite time-steps in order to reach the initial condition $z_0$

* In order to optimize $L$ one needs to compute the gradients wrt. its parameters: $z(t_0), t_0, t_1, \theta$. 

$$ \frac{\partial L(z(t_1))}{\partial \theta}$$

* But, by the chain-rule, one can pretty much lose get lost:

$$ \frac{\partial L(z(t_1))}{\partial \theta} = \frac{\partial L(z(t_1))}{\partial z(t_1)} * 
\frac{\partial z(t_1)}{\partial z(t_1 - \delta t)} * \frac{\partial z(t_1 - \delta t)}{\partial z(t_1 - 2\delta t)} ...
\frac{\partial z(t_0)}{\partial \theta}$$

* Thus, for optimizing $\theta$ one should backpropagate the loss over every $t$ and be able to calculate the product of the every intermediate state. 

* So instead, the paper uses the *adjoint method* for computing the gradients by solving a different (easier, despite the appearances) problem by solving 3 different ODE's.
$$\frac{\partial L}{\partial \theta} = \int_{t_1}^{t_0} a(t) \frac{\partial f(z(t), t, \theta)}{\partial \theta} dt \tag{2.1}$$

$$\frac{\partial L}{\partial z(t)} = -\int_{t_1}^{t_0} a(t) \frac{\partial f(z(t), t, \theta)}{\partial z} dt\tag{2.2}$$

$$z(t_1)=-\int_{t_1}^{t_0} f(z(t), t, \theta) dt\tag{2.3}$$

A simple/intuitive derivation of all those equations goes below..

## Adjoint method (continuous backpropagation in residual networks)

### First equation derivation (equation $2.1$).

* Consider the following 3-steps approximation of a n-ODE-like.

<img src="assets/3-steps.png">

* We want to backpropagate the loss $L(z_3)$ through the network in order to optimize its weights.

* Given that we got shared weights between all timesteps:

$$ \frac{\partial L(z_3)}{\partial \theta} = L(z_2) + L(z_1) + L(z_0)$$

$$= \\
\frac{\partial L}{\partial z_3}\frac{\partial z_3}{\partial f(z_2)}\frac{\partial f(z_2)}{\partial \theta} + \\
\frac{\partial L}{\partial z_2}\frac{\partial z_2}{\partial f(z_1)}\frac{\partial f(z_1)}{\partial \theta} + \\
\frac{\partial L}{\partial z_1}\frac{\partial z_1}{\partial f(z_0)}\frac{\partial f(z_0)}{\partial \theta}
$$

$$ \frac{\partial L}{\partial \theta} = \sum_{n=t_0}^{t_2} \frac{\partial L}{\partial z(t_n)} \frac{\partial f(z_n)}{\partial \theta}$$

* Extending the summation to an integral (approaches to infinite timesteps of infinitely small durations) and considering $t_3 = t_{final}=t_{1}$ 

$$ \frac{\partial L}{\partial \theta} = \int_{t_1}^{t_0} \frac{\partial L}{\partial z(t)} \frac{\partial f(z, t)}{\partial \theta} \tag {3}$$

### Second equation derivation (equation $2.2$).

* For the first term inside the integral in $(3)$, it is needed a way of calculating the gradient of the loss w.r.t. the state $z$ at any time given.
$$ \frac{\partial L}{\partial z(t)} $$ 

$$ \frac{\partial L}{\partial z(t)} = \frac{\partial L}{\partial z(t + \delta t)}\frac{\partial z(t+\delta t)}{\partial z(t)}$$ 

We can use the equation $(2)$ and replace it in $\partial z(t + \delta t)$. It yields:

$$ \frac{\partial L}{\partial z(t)} = \frac{\partial L}{\partial z(t + \delta t)}
\frac{\partial (\int_{t}^{t + \delta t} f(z(t), t, \theta)dt + z(t))}{\partial z(t)}$$ 

I order to simplify the notation, we can call $\frac{\partial L}{\partial z(t)} = \alpha(t)$ (this is called *adjoint state*), thus:

$$ \alpha(t) = \alpha(t + \delta t)
\frac{\partial (\int_{t}^{t + \delta t} f(z(t), t, \theta)dt + z(t))}{\partial z(t)}$$ 

The integral can be approximated by a *Talor Series* of 1st order around z(t): $\int_{t}^{t + \delta t} f(z(t), t, \theta)dt + z(t) \approx z(t) + \delta t f(z(t), t, \theta )$. Using it in the previous equation:



$$ \alpha(t) = \alpha(t + \delta t)  + \alpha(t + \delta t) \delta t \frac{\partial f(z(t), t, \theta )}{\partial z}$$ 

Rearranging:

$$ \frac{\alpha(t + \delta t)-\alpha(t)}{\delta t}  = -\alpha(t + \delta t) \frac{\partial f(z(t), t, \theta )}{\partial z}$$ 

When $\lim_{\delta t\to 0} $, we have:
$$\frac{\partial \alpha(t)}{\partial t} = \alpha(t)\frac{\partial f(z(t), t, \theta )}{\partial z(t)} $$ 

Integrating from $t_1$ to $t_0$ we have:

$$\frac{\partial L}{\partial z(t)} = \int_{t_1}^{t_0} \alpha(t)\frac{\partial f(z(t), t, \theta )}{\partial z(t)} \tag {4}$$ 

Here, the last term of this equation represents the derivative of the "network" function, so, it is easily calculable by definition.

### Third equation derivation (equation $2.3$).

* Well, the third equation is simply a rewrite of the equation $(2)$, the equation of the system.